# 📌 리뷰 데이터 전처리 및 키워드 기반 카테고리 분류

## 🔹 주요 기능
1. **텍스트 전처리 (Preprocessing)**
   - 개행 문자 및 특수문자 제거
   - 불필요한 공백 정리
   - 맞춤법 교정 (Spell Checker)
   - 감탄사 및 불필요한 표현 제거
   - 한글, 영어, 숫자만 유지

2. **문장 단위 분리 및 정제 (Sentence Segmentation & Cleaning)**
   - 리뷰를 문장 단위로 분리
   - 각 문장을 전처리 함수로 정리하여 불필요한 요소 제거

3. **키워드 기반 카테고리 분류 (Keyword-based Categorization)**
   - 특정 키워드(착용감, 사이즈, 내구성 등)에 따라 문장을 해당 카테고리로 분류
   - 키워드가 없을 경우 "기타"로 분류

| **카테고리**        | **설명** |
|--------------------|----------------------------------|
| 착용감            | 편안함, 불편함, 착용 시 느낌에 대한 언급  |
| 사이즈            | 발볼, 발등, 크기 등 사이즈 관련 언급  |
| 내구성 및 품질     | 내구성, 마모, 재질, 퀄리티 관련 언급  |
| 디자인            | 스타일, 트렌드, 색상 등 디자인 관련 언급  |
| 가성비            | 가격 대비 성능, 할인, 저렴함 관련 언급  |
| 배송 및 포장      | 배송 속도, 포장 상태, 택배 서비스 관련 언급  |
| 기타              | 위의 카테고리에 해당하지 않는 문장 |

4. **리뷰 데이터 가공 및 저장 (Processed Data Storage)**
   - 리뷰 데이터를 `[review_id, text_sentence, review_category, rating]` 형태로 변환
   - 향후 분석 및 학습을 위해 저장

In [ ]:
!pip install kss
!pip install konlpy
!pip install handspell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 불용어 리스트 (set 활용으로 속도 높임)
stopwords = set([
    "완전", "진심", "정말로", "진짜루", "진짜로", "너무", "엄청나게", "진짜", "훨씬", "마치", "생각보다", "생각 보다", "상당히", '젤','되게','디게','엄청',
    "그리고", "그래서", "그러나", "하지만", "그런데", "또한", "게다가", "왜냐면", "즉", "결과적으로", "즉슨",'가장',
    "하다", "있다", "되다", "않다", "같다", "없다", "이었다", "했습니다", "했어요", "보였다", "보여요",'제일',
    "무신사", "29cm", "ABC마트", "정품", "리뷰", "구매평", "상품평", "사이트", "링크", "네이버", "카카오", "페이스북",
    "진짜", "정말", "생각", "그냥", "솔직히", "어느 정도", "조금", "조금은", "대충", "뭔가", "이것", "저것", "이건", "저건", "너무", "엄청", "매우",
    '약간','약간의','좀','그래도','뭔가','먼가','꼭','다만','딱','넘','너무','아주','와우','오우','일단','근데','살짝','점점'
])

# 토픽 분류를 위한 키워드 모음
extended_keyword_dict = {
    "착용감": [
        "착용", "편안", "불편", "쿠션", "푹신",'폭신', "딱", "아픔",'아픕', "아파","아프",'아플','아픈', "편함", "편해",'편하', "착화", "장시간",'양말 필수','양말은 필수',
         "가볍", "가벼", "무겁","무거", "통풍", "신축", "탄력", "착 감" , '까져','까지', '반창고','밴드','편한','피로','물집','상처','좁아','좁고','좁네','타이트','데일리',
        '포근','느낌','부드','벗겨','따뜻','따듯','길들','길드','까져','까졌','까졋','까진','까지','저려','무리', '착 화','양말','끼는','낍','까지','까졌','까집','라이트'
    ],
    "사이즈": [
        "사이즈", "사 이즈",'사이주', "크","큽", "작", "발 볼", "길이", "정사이즈", "오버", "딱맞다","발볼",'꽉','큰','벗겨','여유','널널','치수','10단위','10 단위','5단위',
        '5 단위',"헐렁", "슬림", "루즈", '210','215','220','225','230','235','240','245','250','255','260','265','270','275','280','285','290','295','300','업','벗겨',
        '여유','발등','끼'
    ],
    "내구성 및 품질": [
        "내구성", "오래", "튼", "질", "마감", "재질", "해짐", "금방", "벗겨짐", "찢어","퀄리티", "박음","본드","냄새",'닳',
        "스크래치", "먼지", "단단", "마모", "시간지남", "늘어", "터짐", "탄탄",'가죽'
    ],
    "디자인": [
        "디자인", "스타일",'예쁨','이쁨',"예쁜",'이쁜',"예뻐",'이뻐',"예쁘",'이쁘','이뻤','예뻤', "멋짐", "심플", "트렌디", "고급", "유행", "감각", "클래식", "어울", "모양","룩",
        "세련", "귀여운", "차분한", "화려함", "깔끔한", "베이직", "유니크", "섹시", "모던", "주얼", '블랙', '화이트', '흰', '검', '색' ,'핏','쉐입','쉐잎','아이보리',
        '코디','외관','미니','색감','튀','컬러','착장','귀엽','귀여','남성','여성','룩','크기','옷','실물','굽','높이','패션','팬츠','다리','바지','길어',
        '앞','뒤','뒷','이쁩','예쁩','광택','멋','포인트','깔끔','아무 옷','아무옷','어떤 옷','어떤옷','무광','유광','정장'
    ],
    "가성비": [
        "가성비","가성 비", "가격대비", "비싼","비싸", "저렴", "할인", "가격", "돈", "합리적", "질 대비", '블프','세일','입문',
        "싼", "싸게", "싸네", "비싼", "가격만족", "가격적당", "가격부담", "경제적", "가격높음", "퀄리티대비", "가격메리트", "아깝다", "아까운",'득템'
    ],
    "배송 및 포장 및 응대": [
        "배송", "빠름", "느림", "포장", "박스", "찢어짐", "배송", "파손", "안전포장", "택배", '도착','출발','과정',
        "지연", "빠른배송", "포장", "배송빠름", "배송만족", "택배기사", "빠른도착", "파손됨",'불량','응대','친절'
    ]
}

In [ ]:
# 함수 모음

import kss
import re
from hanspell import spell_checker
import pandas as pd

def correct_spelling(text):
    try : return spell_checker.check(text).checked
    except : return text  # 오류 발생 시 원본으로

def preprocess_text(text):
    text = text.replace("\n", " ")  # 개행 문자 제거
    text = re.sub(r"\.{2,}더보기", "", text)
    return correct_spelling(text)  # 맞춤법 교정 추가

def normalize_spaces(text):
    return ' '.join(text.split())

def clean_text_no_morph(text):
    text = re.sub(r"[ㅋㅎㅠㅜㄷ]+", "", text)     # 감탄사 및 불필요한 표현 제거
    words = text.split()  # 띄어쓰기 기준으로 나눔
    words = [word for word in words if word not in stopwords]  # 불용어 제거
    cleaned_text = normalize_spaces(" ".join(words)) # 띄워쓰기 정규화
    cleaned_text = re.sub(r"[^가-힣a-zA-Z0-9\s]", "", cleaned_text) # 숫자, 한글, 영어만 유지 (특수문자 제거)

    return cleaned_text

def process_review_no_morph(review):

    review = preprocess_text(review)  # 불필요한 요소 제거
    sentences = kss.split_sentences(review)  # 문장 분리
    sentences = [clean_text_no_morph(sentence) for sentence in sentences]  # 각 문장에서 정리
    return sentences  # 리스트 형태로 반환

def assign_labels(sentence):
    #각 문장별로 카테고리 분류
    labels = []
    for category, keywords in extended_keyword_dict.items():
        if any(keyword in sentence for keyword in keywords):  # 키워드 포함 여부 확인
            labels.append(category)
    return labels if labels else ["기타"]  # 키워드가 없으면 '기타' 카테고리로 분류

def process_reviews_with_labels(reviews):
    results = []

    for review_id, review_text, rating in reviews:  # CSV에서 불러온 데이터를 처리
        processed_sentences = process_review_no_morph(review_text)  # 전처리 수행

        for sentence in processed_sentences:
            labels = assign_labels(sentence)  # 키워드 매칭 수행
            results.append([review_id, sentence, labels,rating])  # [리뷰ID, 전처리된 문장, 키워드 영역]로 반환

    return results

ModuleNotFoundError: No module named 'hanspell'

# 테스트

In [ ]:
# 실제 활용해보기, 데이터 로드

'''import pymysql

conn = pymysql.connect(
    host="15.152.242.221",
    user="admin",  # 사용자의 MySQL 계정
    password= "Admin@1234",  # MySQL 비밀번호
    database="musinsa_pd_rv",
    charset="utf8mb4"
)

# SQL 쿼리 실행 (rating과 review_text 가져오기)
query = "SELECT rating, review_text FROM reviews WHERE review_text IS NOT NULL AND review_text != ''"
df = pd.read_sql(query, conn)
conn.close() '''

#df = pd.read_csv("G:\내 드라이브\자료\멀티캠퍼스\최종 프로젝트\Reviews.csv")
df = df[1500:2000] #일단 테스트로 500개의 데이터로만 진행, 만약 기능이 완성된다면 전체 데이터로 진행
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 3000 to 3499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    500 non-null    object
 1   product_id   500 non-null    int64 
 2   rating       500 non-null    int64 
 3   review_text  500 non-null    object
 4   review_date  500 non-null    object
 5   review_size  500 non-null    object
dtypes: int64(2), object(4)
memory usage: 23.6+ KB


In [ ]:
#  review_id, review_text,rating 컬럼만 가져오기
review_data = df[['review_id', 'review_text','rating']].values.tolist()
review_data #확인

[['60041401', '슬랙스 바지 등에 신기에 무난합니다\n발이 아직 길이 안들어서 그런가 오래신지는 못하겠네요', 5],
 ['60022817', '너무 이뻐서 재주문하고자 합니다 이쁜제품 감사드려요!', 5],
 ['59758881', '살짝 크긴한데 입을만해요\n색상 마음에 들고 만족합니다', 5],
 ['59671660', '첫 입문 더비인데 완전 편하고 가격대비 완전 훌륭한 것 같아요! 자주 신을 것 같아요😊', 5],
 ['59614714', '사이즈 잘 맞고 어느정도 키높이효과 잇어요. 좋아요', 5],
 ['59389900', '아주조아요 품질도 좋습니다. 깔끔하구용 고마워요', 5],
 ['59327205', '잘사용하고있어요 감사합니다 ㅎㅎ 제품이 사진과 같아요', 5],
 ['58969743', '정 사이즈로 주문하시면 됩니다.\n\n기본적으로 정장 구두라기 보다는 캐주얼 하지만...더보기', 5],
 ['59020127', '저렴하게 좋은 제품 구매 했어요!\n비슷한 가격대에서 최고예요!! 많이 파세요!!', 5],
 ['58984930', '좋아요 색감도 예쁘고 여기저기 매치하기 좋네요.', 5],
 ['58969911', '정 사이즈로 가시면 되고, 캐주얼 하면서\n\n편안한 남성 구두를 찾는다면 대안이 없을 정도로...더보기', 5],
 ['58969814',
  '무신사에서 구매하고 가장 만족하는 제품 중에 하나\n\n기본적으로 편안하면서 캐주얼 하게 입고 싶을 때...더보기',
  5],
 ['58926643', '지금까지 신었던 구두중에 발도 제일 편하고 가성비도 훌류해요', 5],
 ['58926626', '지금까지 신었던 구두중에 발도 제일 편하고 가성비도 훌륭해요', 5],
 ['58922693', '보기보다 너무 가볍고 편해요. 아무 옷에나 다 잘 어울리는 기본템입니다.', 5],
 ['58922465',
  '사이즈는 넉넉한 편이고, 굽이 좀 있는 편인데 반해 너무 가볍고 편합니다. 치노, 셋업, 데님 아무데나 다

In [ ]:
# 리뷰 데이터 전처리 & 키워드 분석 수행
# [[review_id, text_sentence, review_category, rating],[ ], ...]으로 저장
processed_results = process_reviews_with_labels(review_data)

In [ ]:
# 데이터 저장 방식 확인
for row in processed_results : print(row)

['60041401', '슬랙스 바지 등에 신기에 무난합니다', ['디자인'], 5]
['60041401', '발이 아직 길이 안들어서 그런가 오래신지는 못하겠네요', ['사이즈', '내구성 및 품질'], 5]
['60022817', '이뻐서 재주문하고자 합니다', ['디자인'], 5]
['60022817', '이쁜제품 감사드려요', ['디자인'], 5]
['59758881', '크긴한데 입을만해요', ['사이즈'], 5]
['59758881', '색상 마음에 들고 만족합니다', ['디자인'], 5]
['59671660', '첫 입문 더비인데 편하고 가격대비 훌륭한 것 같아요', ['착용감', '가성비'], 5]
['59671660', '자주 신을 것 같아요', ['기타'], 5]
['59614714', '사이즈 잘 맞고 어느정도 키높이효과 잇어요', ['사이즈', '디자인'], 5]
['59614714', '좋아요', ['기타'], 5]
['59389900', '아주조아요', ['기타'], 5]
['59389900', '품질도 좋습니다', ['내구성 및 품질'], 5]
['59389900', '깔끔하구용 고마워요', ['기타'], 5]
['59327205', '잘사용하고있어요', ['기타'], 5]
['59327205', '감사합니다', ['기타'], 5]
['59327205', '제품이 사진과 같아요', ['기타'], 5]
['58969743', '정 사이즈로 주문하시면 됩니다', ['사이즈'], 5]
['58969743', '기본적으로 정장 구두라기 보다는 캐주얼', ['디자인'], 5]
['59020127', '저렴하게 좋은 제품 구매 했어요', ['가성비'], 5]
['59020127', '비슷한 가격대에서 최고예요', ['가성비'], 5]
['59020127', '많이 파세요', ['기타'], 5]
['58984930', '좋아요', ['기타'], 5]
['58984930', '색감도 예쁘고 여기저기 매치하기 좋네요', ['디자인'], 5]
['58

# 테스트

In [ ]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification

# 모델 및 토크나이저 로드
MODEL_PATH = "C:\최종프로젝트\kobert_sentiment_model(25.2.19).pth"
tokenizer = AutoTokenizer.from_pretrained("monologg/kobert")
model = BertForSequenceClassification.from_pretrained("monologg/kobert", num_labels=2)

# 저장된 모델 가중치 불러오기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

# 감정 분석 함수
def predict_sentiment(sentence):
    encoding = tokenizer(
        sentence,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="pt"  # 여기에 .to(device) 적용 X
    )

    # Tensor 데이터를 device로 이동
    input_ids = encoding["input_ids"].to(device)
    attention_mask = encoding["attention_mask"].to(device)

    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=attention_mask)
        probs = torch.nn.functional.softmax(output.logits, dim=-1)
        label = torch.argmax(probs, dim=-1).item()

    sentiment = "긍정 😀" if label == 1 else "부정 😞"
    confidence = probs[0][label].item() * 100

    print(f"📝 입력 문장: {sentence}")
    print(f"📊 예측 결과: {sentiment} (확률: {confidence:.2f}%)")

    return label, confidence


The repository for monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-73f268bdf301>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recomm

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\최종프로젝트\\kobert_sentiment_model(25.2.19).pth'

In [ ]:
import os

MODEL_PATH = "C:\\최종프로젝트\\kobert_sentiment_model(25.2.19).pth"

if os.path.exists(MODEL_PATH):
    print("✅ 파일이 존재합니다.")
else:
    print("❌ 파일이 존재하지 않습니다. 경로를 확인하세요.")

❌ 파일이 존재하지 않습니다. 경로를 확인하세요.


In [1]:
# 🔥 Focal Loss 함수 추가
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2.0):
        super(FocalLoss, self).__init__()
        self.gamma = gamma

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

NameError: name 'torch' is not defined